In [1]:
from pathlib import Path

text = Path('tiny-shakespeare.txt').read_text()

In [2]:
text[0:1000]

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

In [3]:
import torch

class CharTokenizer:
  def __init__(self, vocabulary):
    self.id_for_char = {chr: id for id, chr in enumerate(vocabulary)}
    self.char_for_id = {id: chr for id, chr in enumerate(vocabulary)}

  @staticmethod
  def train_from_text(text):
    vocabulary = set(text)
    return CharTokenizer(vocabulary)

  def encode(self, text):
    ids = []
    for chr in text:
      ids.append(self.id_for_char[chr])
    return torch.tensor(ids, dtype=torch.long)

  def decode(self, ids):
    chars = []
    for id in ids.tolist():
      chars.append(self.char_for_id[id])
    return ''.join(chars)

  def vocabulary_size(self):
    return len(self.id_for_char)

In [4]:
tokenizer = CharTokenizer.train_from_text(text)

In [5]:
print(tokenizer.encode("Hello world"))
print(tokenizer.decode(tokenizer.encode("Hello world")))

tensor([33, 37, 29, 29,  4, 50, 42,  4, 46, 29, 38])
Hello world


In [6]:
print(f"Vocabulary size: {tokenizer.vocabulary_size()}")

Vocabulary size: 65


In [7]:
from torch.utils.data import Dataset

class IndexesDataset(Dataset):
    def __init__(self, data, block_size):
        self.data = data
        self.block_size = block_size

    def __len__(self):
        # Ensure all sequences generated are complete by reducing length
        return len(self.data) - self.block_size

    def __getitem__(self, pos):
        assert pos < len(self.data) - self.block_size

        x = self.data[pos:pos + self.block_size]
        y = self.data[pos + 1:pos + 1 + self.block_size]
        return x, y

In [8]:
tokenized_text = tokenizer.encode(text)
dataset = IndexesDataset(tokenized_text, 64)

In [9]:
from torch.utils.data import DataLoader, RandomSampler

sampler = RandomSampler(dataset, replacement=True)
dataloader = DataLoader(dataset, batch_size=2, sampler=sampler)

In [10]:
x, y = next(iter(dataloader))

In [11]:
x

tensor([[29, 37,  6, 19, 62, 56, 11, 51, 37, 62, 50, 26,  4, 19, 62, 50, 19, 17,
         19,  9, 37, 40, 62, 37, 38, 56, 50,  6, 19, 50, 62, 22, 37, 50, 62, 13,
         26, 37, 50,  6, 12, 38, 50,  9, 29,  6, 40, 37, 11, 35,  4, 62, 22, 50,
         26,  6, 30, 37, 50,  6, 25,  6, 13, 12],
        [26, 44, 50, 29, 13, 47, 37, 32, 62, 13, 26, 37, 50, 46, 37, 13, 25, 12,
         50,  6, 19, 50, 30, 13, 12, 25, 63, 11, 11, 51, 55, 64, 59,  8, 11, 61,
          4, 12, 47, 13, 46, 26, 50, 62, 22, 37, 50, 40, 46,  4, 42, 12, 50, 62,
          4, 50, 26, 37, 50,  6, 12, 38, 50, 62]])

In [12]:
tokenizer.decode(x[0])

'least,\nYet most suspected, as the time and place\nDoth make again'

In [13]:
tokenizer.decode(y[0])

'east,\nYet most suspected, as the time and place\nDoth make agains'